In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import time

# = = =  = = =  = = =

from bs4 import BeautifulSoup
import re

# = = =  = = =  = = =

import json

# = = =  = = =  = = =

with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

# = = =  = = =  = = =

print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/2.product_url.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['[Product] No'] = ['' if product_no == '' else int(product_no) for product_no in df_input['[Product] No']]

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

# = = =  = = =  = = =

def crawler(crawler_tuple):
    global api, list_dict_correct, list_series_error, crawler_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        url_request = '' if crawler_series['[Product] Url'] == '' else f'''https://api.messefrankfurt.com/service/esb_api/exhibitor-service/api/2.1/public/product/zh-CN/{crawler_series['[Product] Url'].removeprefix('https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/').removesuffix('.html')}/AUTOMECHANIKASHANGHAI/details'''

        # = = =  = = =  = = =

        if url_request == '':
            crawler_retry, url_check = 0, ''
            with semaphore_error:
                list_dict_correct.append({'Url': crawler_series['Url']})
        else:
            crawler_retry = 0
            while True:
                crawler_retry += 1

                try:
                    url_check = crawler_series['[Product] Url']

                    resp = requests.get(url_request,
                                        headers=get_header(**{'Apikey': api}),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200 or resp.status_code == 401:
                        dict_ = resp.json()

                        if resp.status_code == 200:
                            if dict_['success'] or dict_['systemErrors'] and dict_['systemErrors'][0]['statusCode'] == 400 and dict_['systemErrors'][0]['description'].strip() == 'FIND error: null':
                                break
                        elif resp.status_code == 401 and dict_['status'] == 401 and dict_['message'].strip() == 'API GW: Unauthorized':
                            print(f'fuck api {api}')
                            time.sleep(3)
                            with open('./api.txt', 'r') as api_txt:
                                api = api_txt.read()
                        elif resp.status_code == 400 and dict_['status'] == 400 and dict_['systemErrors'][0]['description'].strip() == 'FIND error: null':
                            raise
                except KeyboardInterrupt:
                    break
                except:
                    time.sleep(3)
                    continue

            # = = =  = = =  = = =

            if dict_['systemErrors'] and dict_['systemErrors'][0]['statusCode'] == 400 and dict_['systemErrors'][0]['description'].strip() == 'FIND error: null':
                raise

            # = = =  = = =  = = =

            dict_src, list_src = {}, dict_['result']['description']['images']
            for i, src in enumerate(list_src):
                dict_src[str(i)] = f'https://exhibitorsearch.hk.messefrankfurt.com/images/large/{src.strip()}'

            # = = =  = = =  = = =

            if dict_['result']['description']['text']:
                soup = BeautifulSoup(dict_['result']['description']['text'], 'lxml')
                description = re.sub(r'\t+', '\n', soup.get_text()).strip()
            else:
                description = ''

            # = = =  = = =  = = =

            with semaphore_correct:
                list_dict_correct.append({'Url': crawler_series['Url'],
                                          '[Product] No': crawler_series['[Product] No'],
                                          '[Product] Name': dict_['result']['name'].strip(),
                                          '[Product] Description': description,
                                          '[Product] Picture': '',
                                          '[Product] Url': crawler_series['[Product] Url'],
                                          '[Product] Json_Src': json.dumps(dict_src)})

        # = = =  = = =  = = =

        crawler_status = True
    except:
        crawler_series['Check_Url'] = url_check

        with semaphore_error:
            list_series_error.append(crawler_series)


    # = = =  = = =  = = =

    if crawler_remain % 30 == 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {url_check}\n''')

    with semaphore_remain:
        crawler_remain -= 1

# = = =  = = =  = = =

pool.map(crawler, df_input.iterrows())

print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Url', '[Product] No'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/3.product-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
print('Output Finished !')
print()
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Url'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./file/product_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

# = = =  = = =  = = =

print('Done ~')

[True] - [请求1次] - [剩余29条] - [2024/12/02 19:45:42] - https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zibo-sankyo-rikagaku-co-ltd/mf_2_a3pk1a04hc7z_4228776_10000100202401.html

[True] - [请求1次] - [剩余28条] - [2024/12/02 19:45:43] - https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zhz-fuel-pump/mf_2_a6uj9a00t508_3931775_10000100202401.html

[True] - [请求1次] - [剩余27条] - [2024/12/02 19:45:43] - https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zibo-sankyo-rikagaku-co-ltd/mf_2_a3pk1a04hc7z_4228777_10000100202401.html

[True] - [请求1次] - [剩余26条] - [2024/12/02 19:45:44] - https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/zibo-sankyo-rikagaku-co-ltd/mf_2_a3pk1a04hc7z_4228778_10000100202401.html

[True] - [请求1次] - [剩余25条] - [2024/12/02 19:45:45] - https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-c